In [3]:
from sklearn.ensemble import RandomForestRegressor as rf

In [4]:
# import library
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import random

plt.style.use(style='ggplot')
np.random.seed(1234)
random.seed(1234)

In [5]:
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")
submission = pd.read_csv("./data/sample_submission.csv")

In [6]:
all_df = pd.concat([train_df, test_df], axis=0, sort=False).reset_index(drop=True)

In [7]:
from sklearn.preprocessing import LabelEncoder

categories = all_df.columns[all_df.dtypes == "object"]
print(categories)
print(len(categories))

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')
43


In [8]:
for cat in categories:
    le = LabelEncoder()
    all_df[cat].fillna("missing", inplace=True)
    le = le.fit(all_df[cat])
    all_df[cat] = le.transform(all_df[cat])
    all_df[cat] = all_df[cat].astype("category")

In [9]:
all_df["TotalSF"] = all_df["TotalBsmtSF"] + all_df["1stFlrSF"] + all_df["2ndFlrSF"]
all_df["Total_bathrooms"] = (
    all_df["FullBath"]
    + all_df["HalfBath"]
    + all_df["BsmtFullBath"]
    + all_df["BsmtHalfBath"]
)

In [10]:
hasnan_cat = []
for col in all_df.columns:
    tmp_null_count = all_df[col].isnull().sum()
    if (tmp_null_count > 0) & (col != "SalePrice"):
        hasnan_cat.append(col)
        print(col, tmp_null_count)

LotFrontage 486
MasVnrArea 23
BsmtFinSF1 1
BsmtFinSF2 1
BsmtUnfSF 1
TotalBsmtSF 1
BsmtFullBath 2
BsmtHalfBath 2
GarageYrBlt 159
GarageCars 1
GarageArea 1
TotalSF 1
Total_bathrooms 2


In [11]:
all_df[hasnan_cat].describe()

,LotFrontage,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageYrBlt,GarageCars,GarageArea,TotalSF,Total_bathrooms
count,2433.000000,2896.000000,2918.000000,2918.000000,2918.000000,2918.000000,2917.000000,2917.000000,2760.000000,2918.000000,2918.000000,2918.000000,2917.000000
mean,69.305795,102.201312,441.423235,49.582248,560.772104,1051.777587,0.429894,0.061364,1978.113406,1.766621,472.874572,2548.048663,2.439150
std,23.344905,179.334253,455.610826,169.205611,439.543659,440.766258,0.524736,0.245687,25.574285,0.761624,215.394815,804.677866,0.941799
min,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1895.000000,0.000000,0.000000,334.000000,1.000000
25%,59.000000,0.000000,0.000000,0.000000,220.000000,793.000000,0.000000,0.000000,1960.000000,1.000000,320.000000,2000.500000,2.000000
50%,68.000000,0.000000,368.500000,0.000000,467.000000,989.500000,0.000000,0.000000,1979.000000,2.000000,480.000000,2450.000000,2.000000
75%,80.000000,164.000000,733.000000,0.000000,805.500000,1302.000000,1.000000,0.000000,2002.000000,2.000000,576.000000,2991.750000,3.000000
max,313.000000,1600.000000,5644.000000,1526.000000,2336.000000,6110.000000,3.000000,2.000000,2207.000000,5.000000,1488.000000,11752.000000,8.000000


In [12]:
for col in all_df.columns:
    tmp_null_count = all_df[col].isnull().sum()
    if (tmp_null_count > 0) & (col != "SalePrice"):
        print(col, tmp_null_count)
        all_df[col].fillna(all_df[col].median(), inplace=True)

LotFrontage 486
MasVnrArea 23
BsmtFinSF1 1
BsmtFinSF2 1
BsmtUnfSF 1
TotalBsmtSF 1
BsmtFullBath 2
BsmtHalfBath 2
GarageYrBlt 159
GarageCars 1
GarageArea 1
TotalSF 1
Total_bathrooms 2


In [13]:
train_df_le = all_df[~all_df["SalePrice"].isnull()]
test_df_le = all_df[all_df["SalePrice"].isnull()]

train_df_le["SalePrice_log"] = np.log(train_df_le["SalePrice"])
train_X = train_df_le.drop(["SalePrice", "SalePrice_log", "Id"], axis=1)
train_Y = train_df_le["SalePrice_log"]

c:\ProgramData\Anaconda3\envs\kaggle_book\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [14]:
from sklearn.model_selection import KFold

folds = 3
kf = KFold(n_splits=folds)

In [15]:
# import mean_squared_error
from pyexpat import model
from sklearn.metrics import mean_squared_error

models_rf = []
rmses_rf = []
oof_rf = np.zeros(len(train_X))

for train_index, val_index in kf.split(train_X):
    X_train = train_X.iloc[train_index]
    X_vaild = train_X.iloc[val_index]
    y_train = train_Y.iloc[train_index]
    y_valid = train_Y.iloc[val_index]
    model_rf = rf(n_estimators=50, random_state=1234)
    model_rf.fit(X_train, y_train)
    y_pred = model_rf.predict(X_vaild)
    tmp_rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    print(tmp_rmse)
    models_rf.append(model_rf)
    rmses_rf.append(tmp_rmse)
    oof_rf[val_index] = y_pred

0.14041732688138514
0.15352088210494158
0.14194449912040752


In [16]:
sum(rmses_rf) / len(rmses_rf)

0.1452942360355781

In [17]:
test_X = test_df_le.drop(["SalePrice", "Id"], axis=1)

In [18]:
preds_rf = []
for model in models_rf:
    pred = model.predict(test_X)
    preds_rf.append(pred)

In [19]:
preds_array_rf = np.array(preds_rf)
preds_mean_rf = np.mean(preds_array_rf, axis=0)
preds_exp_rf = np.exp(preds_mean_rf)
submission["SalePrice"] = preds_exp_rf

In [20]:
submission.to_csv("./submit/submission_rf.csv", index=False)

In [21]:
import xgboost as xgb

In [22]:
categories = train_X.columns[train_X.dtypes == "category"]
for col in categories:
    train_X[col] = train_X[col].astype("int8")
    test_X[col] = test_X[col].astype("int8")

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_vaild, y_train, y_valid = train_test_split(
    train_X, train_Y, test_size=0.2, random_state=1234, shuffle=True, stratify=None
)

In [24]:
import optuna


def objective(trial):
    xgb_params = {
        "learning_rate": 0.05,
        "seed": 1234,
        "max_depth": trial.suggest_int("max_depth", 3, 16),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.2, 0.9),
        "sublsample": trial.suggest_uniform("sublsample", 0.2, 0.9),
    }

    xgb_train = xgb.DMatrix(X_train, label=y_train)
    xgb_eval = xgb.DMatrix(X_vaild, label=y_valid)
    evals = [(xgb_train, "train"), (xgb_eval, "eval")]
    model_xgb = xgb.train(
        xgb_params,
        xgb_train,
        evals=evals,
        num_boost_round=1000,
        early_stopping_rounds=20,
        verbose_eval=10,
    )

    y_pred = model_xgb.predict(xgb_eval)
    score = np.sqrt(mean_squared_error(y_valid, y_pred))
    return score

In [25]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=50)
study.best_params

[16:18:49] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { sublsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:10.95883	eval-rmse:10.94191
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 20 rounds.
[10]	train-rmse:6.57130	eval-rmse:6.55549
[20]	train-rmse:3.94368	eval-rmse:3.92819
[30]	train-rmse:2.37041	eval-rmse:2.35971
[40]	train-rmse:1.42883	eval-rmse:1.42381
[50]	train-rmse:0.86569	eval-rmse:0.86363
[60]	train-rmse:0.52902	eval-rmse:0.53403
[70]	train-rmse:0.32789	eval-rmse:0.34431
[80]	train-rmse:0.20708	eval-rmse:0.24101
[90]	train-rmse:0.13355	eval-rmse:0.18947
[100]	train-rmse:0.08754	eval-rmse:0.16509
[110]

[I 2023-10-13 16:19:09,004] Trial 0 finished with value: 0.14429166944226696 and parameters: {'max_depth': 15, 'colsample_bytree': 0.6149912327575129, 'sublsample': 0.7909860240067121}. Best is trial 0 with value: 0.14429166944226696.


[16:19:09] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { sublsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:10.95883	eval-rmse:10.94191
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 20 rounds.
[10]	train-rmse:6.57130	eval-rmse:6.55549
[20]	train-rmse:3.94368	eval-rmse:3.92819
[30]	train-rmse:2.37041	eval-rmse:2.35971
[40]	train-rmse:1.42881	eval-rmse:1.42321
[50]	train-rmse:0.86561	eval-rmse:0.86294
[60]	train-rmse:0.52915	eval-rmse:0.53345
[70]	train-rmse:0.32803	eval-rmse:0.34528
[80]	train-rmse:0.20734	eval-rmse:0.24259
[90]	train-rmse:0.13387	eval-rmse:0.19052
[100]	train-rmse:0.08803	eval-rmse:0.16559
[110]

[I 2023-10-13 16:19:10,067] Trial 1 finished with value: 0.14361523734845608 and parameters: {'max_depth': 14, 'colsample_bytree': 0.5814182280978277, 'sublsample': 0.49655835953723326}. Best is trial 1 with value: 0.14361523734845608.


[16:19:10] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { sublsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:10.95883	eval-rmse:10.94191
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 20 rounds.
[10]	train-rmse:6.57130	eval-rmse:6.55558
[20]	train-rmse:3.94368	eval-rmse:3.92826
[30]	train-rmse:2.37056	eval-rmse:2.35932
[40]	train-rmse:1.42902	eval-rmse:1.42305
[50]	train-rmse:0.86606	eval-rmse:0.86322
[60]	train-rmse:0.52957	eval-rmse:0.53417
[70]	train-rmse:0.32870	eval-rmse:0.34412
[80]	train-rmse:0.20886	eval-rmse:0.23998
[90]	train-rmse:0.13792	eval-rmse:0.18673
[100]	train-rmse:0.09591	eval-rmse:0.16097
[110]

...

[16:19:35] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { sublsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:10.95883	eval-rmse:10.94191
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 20 rounds.
[10]	train-rmse:6.57130	eval-rmse:6.55558
[20]	train-rmse:3.94368	eval-rmse:3.92826
[30]	train-rmse:2.37084	eval-rmse:2.35906
[40]	train-rmse:1.42934	eval-rmse:1.42248
[50]	train-rmse:0.86633	eval-rmse:0.86324
[60]	train-rmse:0.52994	eval-rmse:0.53343
[70]	train-rmse:0.32918	eval-rmse:0.34410
[80]	train-rmse:0.20894	eval-rmse:0.24057
[90]	train-rmse:0.13592	eval-rmse:0.18890
[100]	train-rmse:0.08993	eval-rmse:0.16410
[110]

[I 2023-10-13 16:19:36,295] Trial 48 finished with value: 0.14172216538070515 and parameters: {'max_depth': 16, 'colsample_bytree': 0.36137311645443093, 'sublsample': 0.4764085054151604}. Best is trial 19 with value: 0.13519437820748714.


[16:19:36] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { sublsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:10.95883	eval-rmse:10.94191
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 20 rounds.
[10]	train-rmse:6.57130	eval-rmse:6.55549
[20]	train-rmse:3.94368	eval-rmse:3.92819
[30]	train-rmse:2.37041	eval-rmse:2.35971
[40]	train-rmse:1.42883	eval-rmse:1.42381
[50]	train-rmse:0.86569	eval-rmse:0.86363
[60]	train-rmse:0.52902	eval-rmse:0.53398
[70]	train-rmse:0.32806	eval-rmse:0.34381
[80]	train-rmse:0.20877	eval-rmse:0.23939
[90]	train-rmse:0.13844	eval-rmse:0.18712
[100]	train-rmse:0.09786	eval-rmse:0.16237
[110]

[I 2023-10-13 16:19:36,843] Trial 49 finished with value: 0.14141224879052297 and parameters: {'max_depth': 7, 'colsample_bytree': 0.6244752498895471, 'sublsample': 0.21343523881653348}. Best is trial 19 with value: 0.13519437820748714.


{'max_depth': 3,
 'colsample_bytree': 0.42079984564692874,
 'sublsample': 0.45459753965983585}

In [29]:
xgb_params = {
    "learning_rate": 0.05,
    "seed": 1234,
    "max_depth": 3,
    "colsample_bytree": 0.42079984564692874,
    "sublsample": 0.45459753965983585,
}

In [30]:
models_xgb = []
rmses_xgb = []
oof_xgb = np.zeros(len(train_X))

for train_index, val_index in kf.split(train_X):
    X_train = train_X.iloc[train_index]
    X_vaild = train_X.iloc[val_index]
    y_train = train_Y.iloc[train_index]
    y_valid = train_Y.iloc[val_index]
    xgb_train = xgb.DMatrix(X_train, label=y_train)
    xgb_eval = xgb.DMatrix(X_vaild, label=y_valid)
    evals = [(xgb_train, "train"), (xgb_eval, "eval")]
    model_xgb = xgb.train(
        xgb_params,
        xgb_train,
        evals=evals,
        num_boost_round=1000,
        early_stopping_rounds=20,
        verbose_eval=20,
    )
    y_pred = model_xgb.predict(xgb_eval)
    tmp_rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    print(tmp_rmse)
    models_xgb.append(model_xgb)
    rmses_xgb.append(tmp_rmse)
    oof_xgb[val_index] = y_pred

[16:25:50] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { sublsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:10.94953	eval-rmse:10.96903
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 20 rounds.
[20]	train-rmse:3.94250	eval-rmse:3.95505


[40]	train-rmse:1.43059	eval-rmse:1.43994
[60]	train-rmse:0.53326	eval-rmse:0.54271
[80]	train-rmse:0.22249	eval-rmse:0.23648
[100]	train-rmse:0.12820	eval-rmse:0.14860
[120]	train-rmse:0.10317	eval-rmse:0.12781
[140]	train-rmse:0.09521	eval-rmse:0.12163
[160]	train-rmse:0.09051	eval-rmse:0.11857
[180]	train-rmse:0.08704	eval-rmse:0.11692
[200]	train-rmse:0.08405	eval-rmse:0.11588
[220]	train-rmse:0.08173	eval-rmse:0.11507
[240]	train-rmse:0.07927	eval-rmse:0.11452
[260]	train-rmse:0.07737	eval-rmse:0.11427
[280]	train-rmse:0.07514	eval-rmse:0.11393
[300]	train-rmse:0.07347	eval-rmse:0.11356
[320]	train-rmse:0.07179	eval-rmse:0.11323
[340]	train-rmse:0.07025	eval-rmse:0.11312
[360]	train-rmse:0.06904	eval-rmse:0.11296
Stopping. Best iteration:
[349]	train-rmse:0.06966	eval-rmse:0.11290

0.11301485461525754
[16:25:51] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { sublsample } might not be used.

  This may not be accurate due to

In [31]:
sum(rmses_xgb) / len(rmses_xgb)

0.12412241773665961

In [32]:
xgb_test = xgb.DMatrix(test_X)

In [33]:
preds_xgb = []
for model in models_xgb:
    pred = model.predict(xgb_test)
    preds_xgb.append(pred)

In [34]:
preds_array_xgb = np.array(preds_xgb)
preds_mean_xgb = np.mean(preds_array_xgb, axis=0)
preds_exp_xgb = np.exp(preds_mean_xgb)
submission["SalePrice"] = preds_exp_xgb

In [35]:
submission.to_csv("./submit/submission_xgb.csv", index=False)

In [37]:
preds_exp = pd.read_csv("./submit/submission05.csv")
preds_exp = np.array(preds_exp["SalePrice"])

In [38]:
preds_ans = preds_exp_xgb * 0.5 + preds_exp * 0.5

In [39]:
submission["SalePrice"] = preds_ans
submission.to_csv("./submit/submission_ans.csv", index=False)